In [37]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
fine_tune_tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_billsum_model")
fine_tune_model = AutoModelForSeq2SeqLM.from_pretrained("stevhliu/my_awesome_billsum_model")
base_tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
base_model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

In [38]:
from datasets import load_dataset
billsum = load_dataset("billsum", split="ca_test")

In [39]:
billsum = billsum.train_test_split(test_size=0.2)

In [40]:
billsum_test = billsum['train']

In [41]:
prefix = "summarize: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = fine_tune_tokenizer(inputs, max_length=1024, truncation=True)
    labels = fine_tune_tokenizer(text_target=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [42]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

In [43]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=fine_tune_tokenizer, model=fine_tune_model)

In [44]:
import evaluate
rouge = evaluate.load("rouge")

In [49]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = fine_tune_tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = fine_tune_tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != fine_tune_tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [50]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_billsum_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [52]:
trainer = Seq2SeqTrainer(model=fine_tune_model,eval_dataset=tokenized_billsum["test"],data_collator=data_collator,tokenizer=fine_tune_tokenizer,args=training_args,compute_metrics=compute_metrics)
eval = trainer.evaluate()
print(eval)

ValueError: Token is required (write-access action) but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.